# wps_parameters

#### wps_parameters is a process that runs the [parameters](https://rvic.readthedocs.io/en/latest/user-guide/parameters/) module of a VIC streamflow routing model, called [RVIC](https://rvic.readthedocs.io/en/latest/). To get started, first instatiate the client. Here, the client will try to connect to a remote osprey instance using the url parameter.

In [1]:
from birdy import WPSClient
import requests
import os
from datetime import datetime
from wps_tools.utils import copy_http_content
from netCDF4 import Dataset
from tempfile import NamedTemporaryFile
from rvic.core.config import read_config

# Ensure we are in the working directory with access to the data
while os.path.basename(os.getcwd()) != "osprey":
    os.chdir('../')

url = 'https://docker-dev03.pcic.uvic.ca/twitcher/ows/proxy/osprey/wps'
osprey = WPSClient(url)

#### Help for individual processes can be diplayed using the ? command (ex. bird.process?).

In [2]:
# NBVAL_IGNORE_OUTPUT
osprey.parameters?

Signature: osprey.parameters(params_config, np=1, version=True, loglevel='INFO')
Docstring:
Develop impulse response functions using inputs from a configuration file or dictionary

Parameters
----------
params_config : string
    Path to input configuration file or input dictionary
np : integer
    Number of processors used to run job
version : boolean
    Return RVIC version string
loglevel : {'CRITICAL', 'ERROR', 'WARNING', 'INFO', 'DEBUG', 'NOTSET'}string
    Logging level

Returns
-------
output : ComplexData:mimetype:`application/x-netcdf`
    Output Netcdf File
File:      ~/Desktop/osprey/</home/sangwonl/Desktop/osprey/venv/lib/python3.6/site-packages/birdy/client/base.py-2>
Type:      method


#### We can use the docstring to ensure we provide the appropriate parameters.

In [3]:
# run parameters
output = osprey.parameters(
    params_config = "tests/data/configs/parameters_local.cfg"
)

#### Once the process has completed we can extract the results and ensure it is what we expected.

In [4]:
input_config = read_config("tests/data/configs/parameters_local.cfg")

# Expected output metadata determined from the input configuration file
expected = {
    "title": "RVIC parameter file",
    "year": datetime.now().strftime("%Y"),
    "month": datetime.now().strftime("%b"),
    "day": datetime.now().strftime("%d"),
    "RvicPourPointsFile": input_config["POUR_POINTS"]["FILE_NAME"].split("/")[-1],
    "RvicUHFile": input_config["UH_BOX"]["FILE_NAME"].split("/")[-1],
    "RvicFdrFile": input_config["ROUTING"]["FILE_NAME"].split("/")[-1],
    "RvicDomainFile": input_config["DOMAIN"]["FILE_NAME"].split("/")[-1],
}

In [5]:
with NamedTemporaryFile(suffix=".nc", prefix="tmp_copy", dir="/tmp", delete=True) as tmp_file:
    output_data = Dataset(copy_http_content(output.get()[0], tmp_file))

    # Metadata of the output of RVIC parameters module
    metadata = {
        "title": output_data.title,
        "year": output_data.history.split()[5],
        "month": output_data.history.split()[2],
        "day": output_data.history.split()[3],
        "RvicPourPointsFile": output_data.RvicPourPointsFile,
        "RvicUHFile": output_data.RvicUHFile,
        "RvicFdrFile": output_data.RvicFdrFile,
        "RvicDomainFile": output_data.RvicDomainFile,
    }
    assert metadata == expected